In [13]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [14]:
import pickle
import pandas as pd

In [15]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [16]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [17]:
taxi_type = 'yellow'
year = int('2022')
month = int('02')

df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet')

In [18]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [19]:
# standar deviation of predictions
y_pred.std()

5.28140357655334

In [20]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [21]:
df_result = pd.DataFrame(
    {'ride_id': df.ride_id,
    'predictions': y_pred})

In [22]:
!mkdir output

mkdir: cannot create directory ‘output’: File exists


In [23]:
output_file = f'output/predictions_{taxi_type}_{year:04d}-{month:02d}.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [24]:
!ls -lh {output_file}

-rw-rw-r-- 1 diego diego 58M Jun 15 13:56 output/predictions_yellow_2022-02.parquet
